In [1]:
using HDF5
using SparseArrays
using Pkg
Pkg.activate("D:\\Documents\\KU\\Dissertation\\Project4\\ReservoirComputing")

using Plots
# gr()
using LaTeXStrings

  Activating project at `D:\Documents\KU\Dissertation\Project4\ReservoirComputing`


In [2]:
using ReservoirComputing

In [3]:
ENV["JULIA_CONDAPKG_BACKEND"] = "Null"
ENV["JULIA_PYTHONCALL_EXE"] = "C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312\\python.exe"
ENV["PATH"] = "C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312;C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312\\DLLs;" * ENV["PATH"]
ENV["PYTHONPATH"] = raw"D:\Documents\KU\Dissertation\Project4\generalized_hnn\GHNN"


"D:\\Documents\\KU\\Dissertation\\Project4\\generalized_hnn\\GHNN"

In [4]:
using PythonCall

In [5]:
function read_model(model,test_data; leaky = 1.0)
    res_size = read_attribute(model,"res_size")
    if haskey(model["input_matrix"],"I")
        
        I1 = read(model["input_matrix"],"I")
        J1 = read(model["input_matrix"],"J")
        X1 = read(model["input_matrix"],"V")
        input_matrix = sparse(I1,J1,X1)
    else
        input_matrix = read(model["input_matrix"],keys(model["input_matrix"])[1])
    end
    
    if haskey(model["reservoir_matrix"],"I")
        I2 = read(model["reservoir_matrix"],"I")
        J2 = read(model["reservoir_matrix"],"J")
        X2 = read(model["reservoir_matrix"],"V")
        reservoir_matrix = sparse(I2,J2,X2,res_size,res_size)
    else
        reservoir_matrix= read(model["reservoir_matrix"],keys(model["reservoir_matrix"])[1])
    end
        
        
    W_out= read(model,"Wout")
	W_out_rev= read(model,"Wout_rev")

    esn = ESN(test_data, reservoir_matrix ,input_matrix;
            reservoir_driver = RNN(leaky_coefficient = α),washout = wash)
    return esn,W_out,W_out_rev
end

function combine_forward_backward_predict(forward, backward, dict_states)
    Nel = length(dict_state)
    U_GH = [pyconvert(Vector,forward[dict_states[i]].values ) for i in 1:Nel]
    U_GH = hcat(U_GH...)'
    U_GHb = [reverse(pyconvert(Vector,backward[dict_states[i]].values ))[1:end-1] for i in 1:Nel]
    U_GHb = hcat(U_GHb...)'
    UU_GHNN_combined = hcat(U_GHb,U_GH)
    return UU_GHNN_combined
end

function combine_forward_backward_predict_RC(forward, backward, x0)
    RC_combined = hcat(reverse(backward, dims = 2),x0,forward)
    return RC_combined
end

function distance_vector(u1,u2,p = 1)
    d_a = (sum((abs.(u1[1:2]-u2[1:2]).^p)))
    # d_b = sum(abs.(angular_distance.(u1[2],u2[2])).^p)
    return d_a
end

distance_vector (generic function with 2 methods)

In [6]:
np = pyimport("numpy");
pd = pyimport("pandas")

Python: <module 'pandas' from 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\pandas\\__init__.py'>

In [7]:
# using PyCall

ghnn = pyimport("ghnn");

In [8]:


nn_paths = "duffing_200/GHNN/nn_1_infer_eq/";
ghnn_model= ghnn.nets.net_from_dir(nn_paths);
sympnet_path = "duffing_200/SympNet/nn_1/";
sympnet_model =ghnn.nets.net_from_dir(sympnet_path);
henon_path = "duffing_200/HenonNet/nn_1/";
henon_model = ghnn.nets.net_from_dir(henon_path);
dict_state = Dict(1 => "q_A", 2 => "p_A")
dict_ylabel = Dict(1 => "q", 2 => "p")

Dict{Int64, String} with 2 entries:
  2 => "p"
  1 => "q"

In [9]:
fid = h5open("ESN_data_test2.h5", "r");
train_test_file = h5open("rnn_test_ld_duff_w200.h5","r")
train_test_file_back = h5open("rnn_test_ld_duff_w200_back.h5","r")
test1 = read(train_test_file,"test_1")

2×201 Matrix{Float64}:
 -1.3764   -1.26996  -1.15569  …  -1.19683  -1.30881  -1.41141   -1.5
  1.01055   1.11062   1.1683      -1.15229  -1.08032  -0.963923  -0.8

In [10]:
wash = 200;
RC_new_combined = h5open("ESN_data_test2.h5", "r");
RCmodel = keys(RC_new_combined)[14]
QP_esn = RC_new_combined[RCmodel]
alp = read_attribute(QP_esn,"res_driver")
i1 = findlast(",",alp)
α = parse(Float64,alp[i1[1]+1:end-1])
esn_cycle_c,W_out_cycle_c,W_out_cycle_c_back = read_model(QP_esn,test1,leaky = α);
tend = 10
dt = 0.1
Nt = Int(tend/dt+1)


101

In [ ]:
keys(RC_new_combined)[14]

In [11]:
fidN = h5open("LD_output_numerics.h5","r");

x = read_attribute(fidN["numerics2_tend_10_gamma_2"],"xgrid");
y = read_attribute(fidN["numerics2_tend_10_gamma_2"],"ygrid");

In [12]:
plot(x,X[:,1])

UndefVarError: UndefVarError: `X` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
plot(y,Y[1,:])

In [15]:
X = reshape(init_cond[:,1],400,400)
Y = reshape(init_cond[:,2],400,400)


400×400 Matrix{Float64}:
 -0.8  -0.79599  -0.79198  -0.78797  …  0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797  …  0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
  ⋮                                  ⋱                             
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.79198  0.79599  0.8
 -0.8  -0.79599  -0.79198  -0.78797     0.78797  0.7919

In [12]:
init_cond = Array{Float64}(undef,160000,2)
for i in 1:160000
    init_cond[i,:] = read_attribute(train_test_file_back["test_$i"],"(x0,y0)")
end
# read_attribute(train_test_file_back["test_$i"],"(alpha0,phi0)")

In [14]:
_,d = findmin(sum(abs.(init_cond' .- [0.75, -0.75]),dims = 1))
# d = findmin(sum(abs.(init_cond' .- [0.9, -0.6]),dims = 1))

(0.003759398496240851, CartesianIndex(1, 5100))

In [15]:
i = d[2]
# i = 40454
test1 = read(train_test_file_back,"test_$i")
predict_states = create_states(esn_cycle_c.reservoir_driver,test1 ,wash,esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,esn_cycle_c.bias_vector)
UU= esn_cycle_c(Generative(Nt),W_out_cycle_c_back , predict_states[:,end])
# U1 = convert_to_polar(UU)

test1f = read(train_test_file,"test_$i")
predict_statesf = create_states(esn_cycle_c.reservoir_driver,test1f ,wash,esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,esn_cycle_c.bias_vector)
UUf= esn_cycle_c(Generative(Nt),W_out_cycle_c, predict_statesf[:,end])
# U1f = convert_to_polar(UUf)

2×101 Matrix{Float64}:
  0.674673   0.604926   0.538995  …   1.12762   1.0475     0.966343
 -0.716183  -0.678515  -0.640151     -0.7894   -0.809455  -0.810977

In [13]:
xhom = LinRange(-sqrt(2),sqrt(2),161)  
yp = sqrt.(xhom.^2-xhom.^4/2)   

161-element Vector{Float64}:
 0.0
 0.220120594610942
 0.3063889990681775
 0.36926334765567387
 0.4195086411505721
 0.46136764886787884
 0.4970530341422326
 0.5279011637341547
 0.554797260267208
 0.5783664613995827
 ⋮
 0.554797260267208
 0.5279011637341543
 0.4970530341422326
 0.46136764886787884
 0.4195086411505721
 0.3692633476556745
 0.3063889990681775
 0.220120594610942
 0.0

In [14]:
plot_array = Array{Any}(undef,13)
t = -20:0.1:20
dt = 0.1
# u0s = [[-0.8, 0.3], [0.75, -0.75], [-0.75, -0.75]]
u0s = [[xhom[33], yp[33]], [0.75, -0.75], [0.75, -0.75]]
dict_ylabel = Dict(1 => "p", 2 => L"q", 3 => "e", 4 => L"LD/\tau")
# Define line styles and labels, markers 
methods = [
    ("data", :solid,:none,1,4),
    ("SympNet", :dot,:diamond,2,2),
    ("HenonNet",:dashdotdot,:utriangle,3,2),
    ("GHNN",    :dashdot,:square,4,2),
    ("RC",     :dash,:circle,5,2)

]
t_jump = 5
for j in 1:3
# j = 1


    _, d = findmin(sum(abs.(init_cond' .- u0s[j]), dims=1))
    i = d[2]

    test1 = read(train_test_file_back, "test_$i")
    test1f = read(train_test_file, "test_$i")
    data = hcat(test1f,reverse(test1[:,1:end-1], dims = 2));
    predict_states = create_states(esn_cycle_c.reservoir_driver, test1, wash,
                                esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,
                                esn_cycle_c.bias_vector)
    UU = esn_cycle_c(Generative(Nt-1), W_out_cycle_c_back, predict_states[:,end])

    predict_statesf = create_states(esn_cycle_c.reservoir_driver, test1f, wash,
                                    esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,
                                    esn_cycle_c.bias_vector)
    UUf = esn_cycle_c(Generative(Nt-1), W_out_cycle_c, predict_statesf[:,end])

    x0 = np.array(test1f[:,end])
    UU_GHNN_combined  = combine_forward_backward_predict(ghnn_model.predict_path(x0,tend),
                                                        ghnn_model.predict_path(x0,-tend), dict_state)
    UU_SympNet_combined = combine_forward_backward_predict(sympnet_model.predict_path(x0,tend),
                                                            sympnet_model.predict_path(x0,-tend), dict_state)
    UU_Henon_combined = combine_forward_backward_predict(henon_model.predict_path(x0,tend),
                                                        henon_model.predict_path(x0,-tend), dict_state)
    UU_RC_combined = combine_forward_backward_predict_RC(UUf, UU, pyconvert(Vector, x0))
    
    struct_data = [data[:,101:301], UU_SympNet_combined, UU_Henon_combined, UU_GHNN_combined, UU_RC_combined]

    err = Array{Vector{Float64}}(undef,4)
    err_p = Array{Vector{Float64}}(undef,4)
    err_q = Array{Vector{Float64}}(undef,4)
    err_glob =  Array{Vector{Float64}}(undef,5)
    for i in 1:5
        if i!=1
            err[i-1] = [distance_vector(struct_data[i][:,j],struct_data[1][:,j],0.7) for j in 1:size(struct_data[1],2)]
            err_p[i-1] = [struct_data[i][1,j]-struct_data[1][1,j] for j in 1:size(struct_data[1],2)]
            err_q[i-1] = [struct_data[i][2,j]-struct_data[1][2,j] for j in 1:size(struct_data[1],2)]    
            # err_glob
        # else
            # err[i-1] = [distance_vector_cart(struct_data[i][:,j],struct_data[1][:,j],0.7) for j in 1:size(struct_data[i],2)]
        end
        err_glob[i] = [distance_vector(struct_data[i][:,j],struct_data[i][:,j+1],0.7) for j in 1:size(struct_data[i],2)-1]
        
        # p1 = plot_one_line(p1, t[div((size(t)[1]+1),2)-Nt+1:skipt:div((size(t)[1]+1),2)+Nt-1],err[i-1][1:skipt:end], methods_p[i])
    end

    ld_d1 = Vector{Vector{Float64}}(undef,4)
    ld_glob = Vector{Vector{Float64}}(undef,5)
    ld_glob[1] = cumsum(err_glob[1][101:200]+reverse(err_glob[1][1:100]))
    for i in 2:5
        ld_d1[i-1] = cumsum(err[i-1][102:201]+reverse(err[i-1][1:100]))*dt
        ld_glob[i] = cumsum(err_glob[i][101:200]+reverse(err_glob[i][1:100]))*dt^(1-0.7)       
    end
    # === Plotting ===
    for ax1 in 1:4

        p = plot()
        if ax1 < 3
            for i in 1:5
                if i ==1 
                    p = plot!(t[101:301], data[ax1,101:301], color=1, lw=4, label="data")
                else
                    
                    p = plot!(t[101:t_jump:301], struct_data[i][ax1,1:t_jump:end],label=methods[i][1],
                            color=methods[i][4], linestyle=methods[i][2], marker = methods[i][3],
                            lw =  methods[i][5])
                end

            end
            plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
              xlim=(-10,10), ylim=(-2,2), tick_direction=:out)
        elseif ax1 == 3
            for i in 2:5
                    p = plot!(t[202:t_jump:301], ld_d1[i-1][1:t_jump:end]./ld_glob[1][1:t_jump:end],label=methods[i][1],
                            color=methods[i][4], linestyle=methods[i][2], marker = methods[i][3],
                            lw =  methods[i][5])

            end
            plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
              xlim=(0,10), ylim=(0,5), tick_direction=:out)
        else
            for i in 1:5
                    p = plot!(t[202:t_jump:301], ld_glob[i][1:t_jump:end]./t[202:t_jump:301] ,label=methods[i][1],
                            color=methods[i][4], linestyle=methods[i][2], marker = methods[i][3],
                            lw =  methods[i][5])

            end
            plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
              xlim=(0,10), ylim=(2,3), tick_direction=:out)
        end
        
        # p = plot(t[202:401], reverse(test1[ax1,1:end-1]), color=1, lw=4, label="data")
        # plot!(t[1:200], test1f[ax1,1:end-1], color=1, lw=4, label="")
        # p = plot(t[101:301], data[ax1,101:301], color=1, lw=4, label="data")
        # # plot!(t[1:200], test1f[ax1,1:end-1], color=1, lw=4, label="")

        # t_jump = 5

        # for 
        # plot!(t[101:t_jump:301], UU_SympNet_combined[ax1,1:t_jump:end],lw=2.5, label=methods[1][1], color=2, linestyle=methods[1][2],marker = methods[1][3],markersize = 2)
        # plot!(t[101:t_jump:301], UU_Henon_combined[ax1,1:t_jump:end], lw=2.5, label=methods[2][1], color=3, linestyle=methods[2][2],marker = methods[2][3],markersize = 2)
        # plot!(t[101:t_jump:301], UU_GHNN_combined[ax1,1:t_jump:end],  lw=2.5, label=methods[3][1], color=4, linestyle=methods[3][2],marker = methods[3][3],markersize = 2)
        # plot!(t[100:t_jump:302], UU_RC_combined[ax1,1:t_jump:end], lw=2.5, label=methods[4][1], color=5, linestyle=methods[4][2], marker = methods[4][3], markersize = 2)

        # plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
        #       xlim=(-10,10), ylim=(-2,2), tick_direction=:out)

        # vline!([-5, 5], lw=2, label="", color="black", linestyle=:dash)

        # if ax1 == 1 && j == 1
        #     plot!(legend=:topright)
        # end
        # if ax1 == 2 && j == 3
        #     p = plot!(legend = :outerbottom, legendcolumns=5)  # Show legend only in the last plot
        # else
            p = plot!(legend = false)         # Hide legend in other plots
        # end
        if j == 3
            xlabel!(L"t")
        end
        plot_array[(ax1-1)*3+j] = p
        # push!(plot_array, p)
    end
        # dist_vec_GHNN = [distance_vector(data[:,100+i], UU_GHNN_combined[:,i],2) for i in 1:201]
        # p = plot(t[101:t_jump:301], [distance_vector(data[:,100+i], UU_SympNet_combined[:,i],2) for i in 1:201][1:t_jump:end],lw=2.5, label=methods[1][1], color=2, linestyle=methods[1][2],marker = methods[1][3],markersize = 2)
        # plot!(t[101:t_jump:301], [distance_vector(data[:,100+i], UU_Henon_combined[:,i],2) for i in 1:201][1:t_jump:end], lw=2.5, label=methods[2][1], color=3, linestyle=methods[2][2],marker = methods[2][3],markersize = 2)
        # plot!(t[101:t_jump:301], [distance_vector(data[:,100+i], UU_GHNN_combined[:,i],2) for i in 1:201][1:t_jump:end],  lw=2.5, label=methods[3][1], color=4, linestyle=methods[3][2],marker = methods[3][3],markersize = 2)
        # plot!(t[100:t_jump:302], [distance_vector(data[:,100+i], UU_RC_combined[:,i],2) for i in 1:201][1:t_jump:end], lw=2.5, label=methods[4][1], color=5, linestyle=methods[4][2], marker = methods[4][3], markersize = 2)
        # p = plot!(ylabel=L"e",legend = false) 

        # if j == 3
        #     xlabel!(L"t")
        # end
        # # 
        # push!(plot_array, p)
        # p = plot(t[202:401], reverse(test1[ax1,1:end-1]), color=1, lw=4, label="data")
    if j == 3
        # ax1 = 2
        # p = plot(t[101:301], data[ax1,101:301], color=1, lw=4, label="data")
        # # p = plot(t[202:401], reverse(test1[ax1,1:end-1]), color=1, lw=4, label="data")
        # # plot!(t[1:200], test1f[ax1,1:end-1], color=1, lw=4, label="")

        # t_jump = 5
        # plot!(t[101:t_jump:301], UU_SympNet_combined[ax1,1:t_jump:end],lw=2.5, label=methods[2][1], color=2, linestyle=methods[1][2],marker = methods[1][3],markersize = 2)
        # plot!(t[101:t_jump:301], UU_Henon_combined[ax1,1:t_jump:end], lw=2.5, label=methods[3][1], color=3, linestyle=methods[2][2],marker = methods[2][3],markersize = 2)
        # plot!(t[101:t_jump:301], UU_GHNN_combined[ax1,1:t_jump:end],  lw=2.5, label=methods[4][1], color=4, linestyle=methods[3][2],marker = methods[3][3],markersize = 2)
        # plot!(t[100:t_jump:302], UU_RC_combined[ax1,1:t_jump:end], lw=2.5, label=methods[5][1], color=5, linestyle=methods[4][2], marker = methods[4][3], markersize = 2)
        p = plot()
            for i in 1:5
                    p = plot!(t[202:t_jump:301], ld_glob[i][1:t_jump:end]./t[202:t_jump:301],label=methods[i][1],
                            color=methods[i][4], linestyle=methods[i][2], marker = methods[i][3],
                            lw =  methods[i][5])

            end
            plot!(  xlim=(0,10), ylim=(2,3), tick_direction=:out)
        # for i in 1:5
        #     if i ==1 
        #         p = plot!(t[101:301], data[ax1,101:301], color=1, lw=4, label="data")
        #     else

        #         p = plot!(t[101:t_jump:301], struct_data[i][ax1,1:t_jump:end],label=methods[i][1],
        #                   color=methods[i][4], linestyle=methods[i][2], marker = methods[i][3],
        #                   lw =  methods[i][5])
        #     end

        # end
        # plot!(ylabel="",xlim=(-10,10), ylim=(-2,2), tick_direction=:out)
        # p = plot([t[201]],[0.] ,lw=4, label="data", color=1)
        # plot!(t[101:t_jump:301], [distance_vector(data[:,100+i], UU_SympNet_combined[:,i],2) for i in 1:201][1:t_jump:end],lw=2.5, label=methods[1][1], color=2, linestyle=methods[1][2],marker = methods[1][3],markersize = 2)
        # plot!(t[101:t_jump:301], [distance_vector(data[:,100+i], UU_Henon_combined[:,i],2) for i in 1:201][1:t_jump:end], lw=2.5, label=methods[2][1], color=3, linestyle=methods[2][2],marker = methods[2][3],markersize = 2)
        # plot!(t[101:t_jump:301], [distance_vector(data[:,100+i], UU_GHNN_combined[:,i],2) for i in 1:201][1:t_jump:end],  lw=2.5, label=methods[3][1], color=4, linestyle=methods[3][2],marker = methods[3][3],markersize = 2)
        # plot!(t[100:t_jump:302], [distance_vector(data[:,100+i], UU_RC_combined[:,i],2) for i in 1:201][1:t_jump:end], lw=2.5, label=methods[4][1], color=5, linestyle=methods[4][2], marker = methods[4][3], markersize = 2)
        # p = plot!(legend = false) 
        # push!(plot_array, p)
        # vline!([-5, 5], lw=2, label="", color="black", linestyle=:dash)
        p = plot!(legend = :outerbottom, legendcolumns=5, axis = false)
        # push!(plot_array, p)
        plot_array[end] = p
    # if j == 3

    end
end

In [ ]:
size(u0s)

In [17]:

t = -20:0.1:20
dt = 0.1
# u0s = [[-0.8, 0.3], [0.75, -0.75], [-0.75, -0.75]]
u0s = [[xhom[33], yp[33]], [0.75, -0.75], [0.75, -0.75]]
dict_ylabel = Dict(1 => "q-\\hat{q}", 2 => L"q-\\hat{q}", 3 => "e", 4 => L"LD/\tau")
# Define line styles and labels, markers 
# methods = [
#     ("data", :solid,:none,1,4),
#     ("SympNet", :dot,:diamond,2,2),
#     ("HenonNet",:dashdotdot,:utriangle,3,2),
#     ("GHNN",    :dashdot,:square,4,2),
#     ("RC",     :dash,:circle,5,2)

# ]


function plot_results(u0s)
    
    nplot = size(u0s, 1)
    plot_array = Array{Any}(undef,4*nplot + 1)
    for j in 1:nplot
        _, d = findmin(sum(abs.(init_cond' .- u0s[j]), dims=1))
        i = d[2]

        test1 = read(train_test_file_back, "test_$i")
        test1f = read(train_test_file, "test_$i")
        data = hcat(test1f,reverse(test1[:,1:end-1], dims = 2));
        predict_states = create_states(esn_cycle_c.reservoir_driver, test1, wash,
                                    esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,
                                    esn_cycle_c.bias_vector)
        UU = esn_cycle_c(Generative(Nt-1), W_out_cycle_c_back, predict_states[:,end])

        predict_statesf = create_states(esn_cycle_c.reservoir_driver, test1f, wash,
                                        esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,
                                        esn_cycle_c.bias_vector)
        UUf = esn_cycle_c(Generative(Nt-1), W_out_cycle_c, predict_statesf[:,end])

        x0 = np.array(test1f[:,end])
        UU_GHNN_combined  = combine_forward_backward_predict(ghnn_model.predict_path(x0,tend),
                                                            ghnn_model.predict_path(x0,-tend), dict_state)
        UU_SympNet_combined = combine_forward_backward_predict(sympnet_model.predict_path(x0,tend),
                                                                sympnet_model.predict_path(x0,-tend), dict_state)
        UU_Henon_combined = combine_forward_backward_predict(henon_model.predict_path(x0,tend),
                                                            henon_model.predict_path(x0,-tend), dict_state)
        UU_RC_combined = combine_forward_backward_predict_RC(UUf, UU, pyconvert(Vector, x0))
        
        struct_data = [data[:,101:301], UU_SympNet_combined, UU_Henon_combined, UU_GHNN_combined, UU_RC_combined]

        err = Array{Vector{Float64}}(undef,4)
        err_glob =  Array{Vector{Float64}}(undef,5)
        for i in 1:5
            if i!=1
                #err is error with reference
                err[i-1] = [distance_vector(struct_data[i][:,j],struct_data[1][:,j],0.7) for j in 1:size(struct_data[1],2)]
            end
            #err_glob is LD
            err_glob[i] = [distance_vector(struct_data[i][:,j],struct_data[i][:,j+1],0.7) for j in 1:size(struct_data[i],2)-1]
            
        end

        ld_d1 = Vector{Vector{Float64}}(undef,4)
        ld_glob = Vector{Vector{Float64}}(undef,5)
        ld_glob[1] = cumsum(err_glob[1][101:200]+reverse(err_glob[1][1:100]))
        for i in 2:5
            ld_d1[i-1] = cumsum(err[i-1][102:201]+reverse(err[i-1][1:100]))*dt
            ld_glob[i] = cumsum(err_glob[i][101:200]+reverse(err_glob[i][1:100]))*dt^(1-0.7)
        end
        # === Plotting ===
        for ax1 in 1:4 # column

            p = plot()
            if ax1 < 3
                for i in 1:5 # plot each line
                    if i ==1 
                        p = plot!(t[101:301], data[ax1,101:301], color=1, lw=4, label="data")
                    else
                        
                        p = plot!(t[101:t_jump:301], struct_data[i][ax1,1:t_jump:end],label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2], marker = methods[i][3],
                                lw =  methods[i][5], ms =3)
                    end

                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(-10,10), ylim=(-2,2), tick_direction=:out)
            elseif ax1 == 3
                # plot 3rd column:  err/LD
                for i in 2:5
                        p = plot!(t[202:t_jump:301], ld_d1[i-1][1:t_jump:end]./ld_glob[1][1:t_jump:end],label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2], marker = methods[i][3],
                                lw =  methods[i][5], ms =3)

                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(0,10), ylim=(0,5), tick_direction=:out)
            else
                # plot 4th column: LD/t
                for i in 1:5
                        p = plot!(t[202:t_jump:301], ld_glob[i][1:t_jump:end]./t[202:t_jump:301] .* (dt^(1-0.7)),label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2], marker = methods[i][3],
                                lw =  methods[i][5], ms = 3)

                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(0,10), ylim=(1,3), tick_direction=:out)
            end
            
            
                p = plot!(legend = false)         # Hide legend in other plot
            if j == nplot
                if ax1 < 3
                    xlabel!(L"t")
                else
                    xlabel!(L"\tau")
                end
            end
            # plot_array[(ax1-1)*nplot+j] = p
            plot_array[(j-1)*4+ax1] = p
        end

        if j == nplot
            p = plot()
                for i in 1:5
                        p = plot!(t[202:t_jump:301], ld_glob[i][1:t_jump:end]./t[202:t_jump:301] ,label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2], marker = methods[i][3],
                                lw =  methods[i][5], ms = 3)

                end
                plot!(  xlim=(0,10), ylim=(1,3), tick_direction=:out)

            p = plot!(legend = :outerbottom, legendcolumns=5, axis = false)

            plot_array[end] = p


        end
    end
    return plot_array
end
    


plot_results (generic function with 1 method)

In [27]:
dt^(0.7)

0.199526231496888

In [28]:
dict_ylabel = Dict(1 => "q- \\hat{q}", 2 => "p- \\hat{p}", 3 => "e", 4 => L"LD/\tau")
methods = [
    ("data", :solid,:none,1,5.5),
    ("SympNet", :dot  ,:diamond,2,2.5),
    ("HenonNet",:dashdotdot,:utriangle,3,2.5),
    ("GHNN",    :dashdot,:square,4,2.5),
    ("RC",   :dash,:circle,5,2.5)

]
t_jump = 5
t = -20:0.1:20
function plot_results_error(u0s)
    
    nplot = size(u0s, 1)
    plot_array = Array{Any}(undef,4*nplot + 1)
    for j in 1:nplot
        _, d = findmin(sum(abs.(init_cond' .- u0s[j]), dims=1))
        i = d[2]

        test1 = read(train_test_file_back, "test_$i")
        test1f = read(train_test_file, "test_$i")
        data = hcat(test1f,reverse(test1[:,1:end-1], dims = 2));
        predict_states = create_states(esn_cycle_c.reservoir_driver, test1, wash,
                                    esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,
                                    esn_cycle_c.bias_vector)
        UU = esn_cycle_c(Generative(Nt-1), W_out_cycle_c_back, predict_states[:,end])

        predict_statesf = create_states(esn_cycle_c.reservoir_driver, test1f, wash,
                                        esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,
                                        esn_cycle_c.bias_vector)
        UUf = esn_cycle_c(Generative(Nt-1), W_out_cycle_c, predict_statesf[:,end])

        x0 = np.array(test1f[:,end])
        UU_GHNN_combined  = combine_forward_backward_predict(ghnn_model.predict_path(x0,tend),
                                                            ghnn_model.predict_path(x0,-tend), dict_state)
        UU_SympNet_combined = combine_forward_backward_predict(sympnet_model.predict_path(x0,tend),
                                                                sympnet_model.predict_path(x0,-tend), dict_state)
        UU_Henon_combined = combine_forward_backward_predict(henon_model.predict_path(x0,tend),
                                                            henon_model.predict_path(x0,-tend), dict_state)
        UU_RC_combined = combine_forward_backward_predict_RC(UUf, UU, pyconvert(Vector, x0))
        
        struct_data = [data[:,101:301], UU_SympNet_combined, UU_Henon_combined, UU_GHNN_combined, UU_RC_combined]

        err = Array{Vector{Float64}}(undef,4)
        err_p = Array{Vector{Float64}}(undef,4)
        err_q = Array{Vector{Float64}}(undef,4)
        err_glob =  Array{Vector{Float64}}(undef,5)
        for i in 1:5
            if i!=1
                #err is error with reference
                err[i-1] = [distance_vector(struct_data[i][:,j],struct_data[1][:,j],0.7) for j in 1:size(struct_data[1],2)]
                err_p[i-1] = [struct_data[i][1,j]-struct_data[1][1,j] for j in 1:size(struct_data[1],2)]
                err_q[i-1] = [struct_data[i][2,j]-struct_data[1][2,j] for j in 1:size(struct_data[1],2)]
            end
            #err_glob is LD
            err_glob[i] = [distance_vector(struct_data[i][:,j],struct_data[i][:,j+1],0.7) for j in 1:size(struct_data[i],2)-1]
            
        end

        ld_d1 = Vector{Vector{Float64}}(undef,4)
        ld_glob = Vector{Vector{Float64}}(undef,5)
        ld_glob[1] = cumsum(err_glob[1][101:200]+reverse(err_glob[1][1:100]))*dt^(1-0.7)
        for i in 2:5
            ld_d1[i-1] = cumsum(err[i-1][102:201]+reverse(err[i-1][1:100]))*dt
            ld_glob[i] = cumsum(err_glob[i][101:200]+reverse(err_glob[i][1:100]))*dt^(1-0.7)
        end
        # === Plotting ===
        for ax1 in 1:4 # column

            p = plot()
            if ax1 ==1
                for i in 2:5 # plot each line
                    if i ==1 
                        p = plot!(t[101:301], data[ax1,101:301], color=1, lw=4, label="data")
                    else
                        
                        p = plot!(t[101:t_jump:301], err_p[i-1][1:t_jump:end],label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2],
                                #  marker = methods[i][3],
                                lw =  methods[i][5], ms =3)
                    end

                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(-10,10),
                 ylim=(-1,1), 
                 tick_direction=:out)
            elseif ax1 == 2

                for i in 2:5 # plot each line
                    if i ==1 
                        p = plot!(t[101:301], data[ax1,101:301], color=1, lw=4, label="data")
                    else
                        
                        p = plot!(t[101:t_jump:301], err_q[i-1][1:t_jump:end],label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2], 
                                # marker = methods[i][3],
                                lw =  methods[i][5], ms =3)
                    end

                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(-10,10),
                 ylim=(-1,1),
                  tick_direction=:out)
            elseif ax1 == 3
                # plot 3rd column:  err/LD
                for i in 2:5
                        p = plot!(t[202:t_jump:301], ld_d1[i-1][1:t_jump:end]./ld_glob[1][1:t_jump:end],label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2],
                                #  marker = methods[i][3],
                                lw =  methods[i][5], ms =3)

                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(0,10), ylim=(0,1), tick_direction=:out)
            else
                # plot 4th column: LD/t
                for i in 1:5
                        p = plot!(t[202:t_jump:301], ld_glob[i][1:t_jump:end]./t[202:t_jump:301] ,label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2],
                                #  marker = methods[i][3],
                                lw =  methods[i][5], ms = 3)

                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(0,10), ylim=(1,3), tick_direction=:out)
            end
            
            
                p = plot!(legend = false)         # Hide legend in other plot
            if j == nplot
                if ax1 < 3
                    xlabel!(L"t")
                else
                    xlabel!(L"\tau")
                end
            end
            # plot_array[(ax1-1)*nplot+j] = p
            plot_array[(j-1)*4+ax1] = p
        end

        if j == nplot
            p = plot()
                for i in 1:5
                        p = plot!(t[202:t_jump:301], ld_glob[i][1:t_jump:end]./t[202:t_jump:301] ,label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2],
                                #  marker = methods[i][3],
                                lw =  methods[i][5], ms = 3)

                end
                plot!(  xlim=(0,10), ylim=(1,3), tick_direction=:out)

            p = plot!(legend = :outerbottom, legendcolumns=5, axis = false)

            plot_array[end] = p


        end
    end
    return plot_array
end
    


plot_results_error (generic function with 1 method)

In [14]:
t[202:t_jump:301]

UndefVarError: UndefVarError: `t` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [30]:
u0s = [[-0.71,-0.51],[-0.71,-0.61],  [-0.71,-0.71]]
pl_err_array = plot_results_error(u0s)
l = @layout [grid(3,4); a{0.085h}]
plot(pl_err_array..., layout = l,size = (1200,500),dpi = 300, left_margin = 4Plots.mm)
savefig("time_trace_homoclinic_Duffing_err.svg")

GKS: Rectangle definition is invalid in routine SET_VIEWPORT
GKS: Rectangle definition is invalid in routine SET_VIEWPORT


"d:\\Documents\\KU\\Dissertation\\Project4\\Duffing\\time_trace_homoclinic_Duffing_err.svg"

In [32]:
u0s = [[-0.71,0.51],[-0.71,0.61],  [-0.71,0.71]]
pl_err_array = plot_results_error(u0s)
l = @layout [grid(3,4); a{0.085h}]
plot(pl_err_array..., layout = l,size = (1200,500),dpi = 300, left_margin = 4Plots.mm)
savefig("time_trace_homoclinic_Duffing_err_plus.svg")

GKS: Rectangle definition is invalid in routine SET_VIEWPORT
GKS: Rectangle definition is invalid in routine SET_VIEWPORT


"d:\\Documents\\KU\\Dissertation\\Project4\\Duffing\\time_trace_homoclinic_Duffing_err_plus.svg"